# PoC: WeCookio multiagent Agno Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Sequential without delegation

### Installing Required Packages with uv

In [ ]:
# Install required packages using uv
!uv add ipykernel agno==1.7.9 boto3 botocore pyyaml ipywidgets prompt-template

### Import Libraries and Load Configuration

In [ ]:
import yaml
import logging
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.aws import AwsBedrock
from agno.utils.pprint import pprint_run_response

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


### Loading AWS Enviroment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [ ]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "agno/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    agno_config = yaml.safe_load(file)
with open(prefix+'config/agents-sequential.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-sequential.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(agno_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

###  Model Selection based on complexity

In [ ]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

### Create Agents with Model-Specific Configuration

In [ ]:
def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = AwsBedrock(
        id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args['role']=agent_config.get("role", None)
    args['goal']=agent_config.get("goal", None)
    args['instructions']=agent_config.get("prompt", None)
    args["description"]=agent_config.get("backstory", None)
    args["name"]=agent_config.get("name", None)
    
    return Agent(**args)

def print_agent_info(agent) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model}\n"
        f"Role: {agent.role}\n"
        f"Goal: {agent.goal}\n"
        f"Instructions: {agent.instructions}\n"
    )
    print("\n")

In [ ]:
# Global metrics storage
def reset_execution_metrics():
    global execution_metrics
    execution_metrics = {
        'audio_tokens': [0],
        'cache_write_tokens': [0],
        'cached_tokens': [0],
        'completion_tokens': [0],
        'input_audio_tokens': [0],
        'input_tokens': [0],
        'output_audio_tokens': [0],
        'output_tokens': [0],
        'prompt_tokens': [0],
        'reasoning_tokens': [0],
        'time': [0],
        'total_tokens': [0]
    }
    return execution_metrics

def get_execution_metrics():
    return execution_metrics

def set_execution_metrics(kwargs):
    """set metrics from kwargs"""
    try:
        execution_metrics['audio_tokens'][0] += kwargs.get('audio_tokens')[0]
        execution_metrics['cache_write_tokens'][0] += kwargs.get('cache_write_tokens')[0]
        execution_metrics['cached_tokens'][0] += kwargs.get('cached_tokens')[0]
        execution_metrics['completion_tokens'][0] += kwargs.get('completion_tokens')[0]
        execution_metrics['input_audio_tokens'][0] += kwargs.get('input_audio_tokens')[0]
        execution_metrics['input_tokens'][0] += kwargs.get('input_tokens')[0]
        execution_metrics['output_audio_tokens'][0] += kwargs.get('output_audio_tokens')[0]
        execution_metrics['output_tokens'][0] += kwargs.get('output_tokens')[0]
        execution_metrics['prompt_tokens'][0] += kwargs.get('prompt_tokens')[0]
        execution_metrics['reasoning_tokens'][0] += kwargs.get('reasoning_tokens')[0]
        execution_metrics['time'][0] += kwargs.get('time')[0]
        execution_metrics['total_tokens'][0] += kwargs.get('total_tokens')[0]
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
    return execution_metrics

In [ ]:
reset_execution_metrics()
set_execution_metrics({'audio_tokens': [0],
 'cache_write_tokens': [10000000],
 'cached_tokens': [0],
 'completion_tokens': [45],
 'input_audio_tokens': [0],
 'input_tokens': [2165],
 'output_audio_tokens': [0],
 'output_tokens': [1000],
 'prompt_tokens': [2],
 'reasoning_tokens': [0],
 'time': [3.657210035999924],
 'total_tokens': [3165]})


print(get_execution_metrics()) 

In [ ]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(agno_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent)


### Create a Flow

In [ ]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Leche",
        "huevos",
        "ternera",
        "maiz",
        "Gluten",
        "arroz",
        "ajo"
    ],
    "exclusions": ["Pimiento"],
    "preferences": [""],
    "output_language": "spanish",
    "country": "Spain"
}
initial_recipe={}
initial_recipe["food_name"]=recipe_request.pop("food_name")
initial_recipe["country"]=recipe_request["country"]
initial_recipe["output_language"]=recipe_request["output_language"]
initial_recipe["servings"]=recipe_request["servings"]


In [ ]:
## Support functions
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt
    

In [ ]:
# Sequential workflow processing
tasks = {
    "initial_recipe_analysis": {
        "description": f'''{variable_injection(tasks_config['Initial Recipe Analysis'], initial_recipe)}''',
        "status": "pending",
        "agent": culinary_coordinator_agent,
        "dependencies": []
    },
    "ingredient_substitution_strategy": {
        "description": f'''{variable_injection(tasks_config['Ingredient Substitution Strategy'], recipe_request)}''',
        "status": "pending",
        "agent": ingredient_substitution_expert_agent,
        "dependencies": ["initial_recipe_analysis"]
    },
    "culinary_experience_enhancement": {
        "description": f'''{variable_injection(tasks_config['Culinary Experience Enhancement'], recipe_request)}''',
        "status": "pending",
        "agent": culinary_experience_optimizer_agent,
        "dependencies": ["ingredient_substitution_strategy"]
    },
    "final_quality_assurance": {
        "description": f'''{variable_injection(tasks_config['Final Quality Assurance'], recipe_request)}''',
        "status": "pending",
        "agent": quality_assurance_chef_agent,
        "dependencies": ["culinary_experience_enhancement"]
    }
}


In [ ]:

from pprint import pprint

# Sequential workflow processing
def weCookio_process_workflow(tasks:dict):
    print(f''' Flow:\n {tasks} \n ''')
    context = ''
    reset_execution_metrics()
    for task in tasks:
        if tasks[task]['dependencies']:
            print(f'''\nInjecting dependency: {tasks[task]['dependencies']}''')
            task_result = tasks[task]['agent'].run(f'''Previous task results:{context}\n\n Task: \n {tasks[task]['description']}''')
        else:
            print(f'''Injecting task: {tasks[task]['description']}\n''')
            task_result = tasks[task]['agent'].run(f'''Task: \n {tasks[task]['description']}''')
        tasks[task]['result'] = task_result
        context = "".join(task_result.content)
        tasks[task]['satus'] = 'done'
        set_execution_metrics(task_result.metrics)
        print(context)
    return tasks        
    

### Run the Flow

In [ ]:
try:
    result = weCookio_process_workflow(tasks=tasks)
except Exception as e:
    print(f"Error running Workflow: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")


In [ ]:
from pprint import pprint

#print(result['final_quality_assurance']['result'].content))
pprint_run_response(result['final_quality_assurance']['result'], markdown=True)
print(f'''\n\n Last Execution Metrics: \n\n''')
pprint(result['final_quality_assurance']['result'].metrics)
print(f'''\n\n Accumulated Execution Metrics: \n\n''')
pprint(get_execution_metrics())
